In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import sys
%matplotlib inline
import scipy.stats as st

# Import API key

with open('/Users/tmlun/Downloads/api_keys.py', 'r') as g_key:
    mykey=g_key.read()
    mykey=mykey.split('=')
    mykey=mykey[1].strip()   
g_key = mykey.strip('"') 
gmaps.configure(api_key=g_key)

# Data File path (CSV)
#data = "./output_data/cities.csv"

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [8]:
# Study data files
RMS_path = "./RMS.csv"
# Read the data
Crime_df = pd.read_csv(RMS_path)
print(Crime_df.head())


  ARREST_DATE      DATE  YEAR    MONTH  WEEK SEX RACE ETHNIC  FBI      SID  \
0    1/1/2016  1/1/2016  2016  JANUARY     1   F    W      N  NaN      NaN   
1    1/1/2016  1/1/2016  2016  JANUARY     1   M    W      H  NaN      NaN   
2    1/1/2016  1/1/2016  2016  JANUARY     1   M    W      H  NaN   664565   
3    1/1/2016  1/1/2016  2016  JANUARY     1   M    W      H  NaN  1049997   
4    1/1/2016  1/1/2016  2016  JANUARY     1   F    W      H  NaN   964686   

   ... INVOLVEMENT  INCIDENT_NUM OFFENSE_CODE               OFFENSE-DESC  \
0  ...           A  SAPD16000022       420001  TABC-Liquor Law Violation   
1  ...           A  SAPD16000026       420001  TABC-Liquor Law Violation   
2  ...           A  SAPD16056633           56               Other Crimes   
3  ...           A  SAPD16000052       131352            Family Violence   
4  ...           A  SAPD16000030       131352            Family Violence   

                       RMS_DESCRIPTION GIS_ARCTYPE DISTRICT              C

In [10]:
# of zipcodes
Zipcodes_df = Crime_df.loc[:,["ZIP"]]

print(Zipcodes_df)

          ZIP
0       78205
1       78201
2       78218
3       78201
4       78207
...       ...
219219  78220
219220  78220
219221  78222
219222  78257
219223  78228

[219224 rows x 1 columns]


In [15]:
Num_of_Crimes_grp = Crime_df.groupby("ZIP")
Num_of_Crimes = Num_of_Crimes_grp["INCIDENT_NUM"].count()
print(Num_of_Crimes)

ZIP
0        4
1        3
11111    6
50701    2
6278     6
        ..
78291    1
78404    1
78648    1
79502    3
81507    1
Name: INCIDENT_NUM, Length: 113, dtype: int64


In [ ]:
Zipcodes_df["Lat"] = ""
Zipcodes_df["Lon"] = ""

# Target city
target_zip = Zipcodes_df["ZIP"].astype(string)

params = {
    "location": "39.952583,-75.16522",  # philadelphia coords,
    "rankby": "distance",
    "type": "postal",
    "key": gkey,
}

# Build the endpoint URL
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?/address=')


for index,row in Zipcodes_df.iterrows():

    # get restaurant type from df
    Zipcode = row['ZIP']

    # add keyword to params dict
    #params['keyword'] = Zipcode

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {Zipcode}.")
    New_url = target_url+str(Zipcode)+
    response = requests.get(New_url,key=g_key).json()
    #print('\n', 'response is of datatype ', type(response), '\n')
    
    # extract results
    Zipcodes_df.loc[index, 'Lat'] = response['results'][0]["geometry"]["location"]["lat"]
    Zipcodes_df.loc[index, 'Lon'] = response['results'][0]["geometry"]["location"]["lng"]
    
   

In [ ]:


target_zip = Zipcodes_df["ZIP"]

params = {
    "location": "39.952583,-75.16522",  # philadelphia coords,
    "rankby": "distance",
    "type": "postal",
    "key": gkey,
}
params = {"address": target_zip, "key": gkey}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

#print("Drill #1: The Geocoordinates of Seattle, WA")

# Run request
response = requests.get(base_url, params=params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

# Convert to JSON
seattle_geo = response.json()

# Extract lat/lng
lat = seattle_geo["results"][0]["geometry"]["location"]["lat"]
lng = seattle_geo["results"][0]["geometry"]["location"]["lng"]

# Print results
print(f"{target_city}: {lat}, {lng}")

In [16]:
locations = Zipcodes_df.astype(float)
Crime = Num_of_Crimes.astype(float)

ValueError: could not convert string to float: '78207-3170'

In [ ]:
fig= gmaps.figure(center=[0,0], zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=100,
                                point_radius=5)

fig.add_layer(heat_layer)

fig